In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# ! pip install iso3166
# ! pip install LangCodes

In [0]:
import pandas as pd
import numpy as np
import os
import json
import re
import os
# from iso3166 import countries
# import langcodes

#### 1.  Identify potential problems with the data:
 
*   Evaluate the source.
    *   We received our data straight from the app database, so it is a pretty reliable source of data.
    * The issues with this database are the format with which the data was stored and exported. 
    * We recieved files which were somewhat like json in nature, but we had to format the json first.
    * We then had to import the json files and format them for use with pandas
*   What makes you think that?  
    *    Data from the app is structured and coerced to be in a certain format with standard answers for all users.
*   Evaluate potential biases and other problems with the data (e.g. political data gathered by a particular party trying to get a particular result, or positive pharmaceutical data from the company selling the pharmaceutical) What will you do to deal with these problems?
    * Potential bias could come from limited data as the app is new, and the user base for the app could be skewed with the newness of the community. Since users of the app are all learning English, there might also be a potential bias in the variety of users.
* Are there missing results or other things that look wrong?
    * We dropped a few rows that were stored as test data before the app had users but besides that, if the data has missing values, it is in a reasonable column/entry. The data is in good shape.
 
#### 2.   What are the strengths and weaknesses of using this data set for answering the questions in the proposal.  If it is not suitable, consider adjusting the questions or acquiring new data.
* Strengths:
    * The features needed for the task of the project were not extremely complex. Due to this and the fact that the data was collected through automation, minimal errors were encountered in the cleaning process.
    * Another strength is that we have data on interactions within a user base of around 300,000. This size will allow us to try out and use a significant amount of features for predicting friendship links and retention.
    * The data we currently have is rich for feature extraction. We have text, images, and video encapsulating user interaction which could be mined for information through NLP and Computer Vision. Since there could be many factors influencing why someone is not retained in the app, we may to use these techniques to infer more about the person to better predict and understand why they leave.
    
 
* Weaknesses:
    * Time series analysis assumes patterns of past can be used to infer patterns of future. This assumption may not be true with the type of community described by our data. Specifically, our data encompases the initial growth of the community which has many obviously inherent biases such as who initially downloaded the app and how much their intents lined up with the that of the app. Perhaps there exists a different set of initial users that would result in a more successful cascade of naturally forming friendships. Such could render all time series data of the past irrelevant in predicting the strongest factors of friendship in the future.
    * In the case of time series data being relevant, there are some data that lack time stamps which could be a significant loss.
    * For obvious reasons, personal data such as culture, religion, and interests, often used in community analysis, is not available to us. This will make it more difficult to predict community interactions.
    * We lack data on lasting relationships from students to teachers and so we will have to create an evolving model with regards to suggesting teachers to students. 
    
#### 3. Revise any other aspects of your proposal in light of what you have learned from examining the data.
* 
    * After examining the data, it appears that we have enough to still answer the questions we proposed, thus no changes to our proposal are necessary at this time.



In [0]:
# The file directories
folder = '/content/drive/My Drive/Hallo_Data/'
clean_folder = folder + 'clean_data/'

In [0]:
def clean_json_files():
    """
    This code takes the json file we get from Google BigQuery,
    removes the last two entries in the JSON file, and  
    converts & writes to a readable json format.
    """
    for f in os.listdir(folder):
        # Read in each of the JSON files 
        if f[-5:] == '.json' and not os.path.exists(
                                        clean_folder + f[:-5] + '.json'):
            # Replace new lines with commas and remove the 
            # last two entries of each JSON entry
            with open(folder + f, 'r') as infile:
                test = infile.read().replace('\n', ',')
                test = test.replace(",\"__error__\":[],\"__has_error__\":false"
                                    , "")

            # Rewrite the readable JSON files as {name}.json
            with open(clean_folder + f[:-5] + '.json', 'x') as outfile:
                outfile.write('[' + test[:-1] + ']')

In [0]:
def get_bans_df(file_name):
    """
    gets the tables of bans
    """
    name = clean_folder+file_name

    if os.path.exists(name+".pkl"):
        return pd.read_pickle(name+".pkl")
    
    # load_from text_file
    with open(name+".txt" ,'r') as file:
        t = file.read().strip().split("s~hallotest-53286rL")
    len(t)

    # define regex to pull data out
    user = re.compile(r"banned-users\" ([^\s]*) ")
    ban = re.compile(r"bans\" ([^\s]*) ")
    desc = re.compile(r"description  (.*) adminName")
    viol = re.compile(r"conductViolations  ([^\s]*) ")

    regex = [user,ban,desc,viol]

    # define columns
    X = []
    cols = ['user_id','ban_id','description']

    # extract data from each row
    for row in t[1:]:
        data = []

        for reg in regex:
            d = user.findall(row)
            if d:
                data.append(d[0])
            else:
                data.append(np.nan)

        violations = viol.findall(row)
        if violations:
            data.append(", ".join(viol.findall(row)))
        else:
            data.append(np.nan)

        X.append(data)

    # create dataframe
    X = np.array(X)
    df = pd.DataFrame(X, columns=cols)
    df.to_pickle(name+".pkl")

    return df

In [0]:
def get_calls_df(file_name='calls'):
    """
    gets the tables of bans
    """
    name = clean_folder+file_name

    if os.path.exists(name+".pkl"):
        return pd.read_pickle(name+".pkl")
  
    # load_from text_file
    with open(name+".txt" ,'r') as file:
        t = file.read().strip().split("s~hallotest-53286r")

    # define regex to pull data out
    call_id =re.compile(r"calls\" ([^\s]*) ")
    user = re.compile(r"fromUserId  ([^\s]*),")
    to_user = re.compile(r"toUserId  ([^\s]*) ")

    c_type = re.compile(r"callType  ([^\s]*)z")
    status = re.compile(r"status  ([^\s]*)z")

    regex = [call_id,user,to_user,c_type,status]

    # define columns
    X = []
    cols = ['call_id','user_id','to_user_id','video','status']

    # extract data from each row
    for row in t[1:]:
        data = []

        for reg in regex:
            d = reg.findall(row)
            # print(d)
            if d:
                data.append(d[0])
            else:
                data.append(np.nan)

        X.append(data)

    # create dataframe
    X = np.array(X)
    df = pd.DataFrame(X, columns=cols)

    # convert to proper types

    # 1 if accepted, 0 otherwise
    clean_status = lambda x: 1 if x == 'accepted' else 0 
    df.status = df.status.apply(clean_status)

    # 1 if video, 0 if audio
    clean_type = lambda x: 0 if x == 'audio' else 1
    df.video = df.video.apply(clean_type)

    df.to_pickle(name+".pkl")

    return df

In [0]:
def get_followers_df(file_name,cols):
    """
        loads in follower and followed_livestreams json 
        file and converts it to pandas
    """
    f = clean_folder +  file_name + '.json'
    pkl_file = clean_folder +  file_name + '.pkl'

    # check for the pickled file
    if os.path.exists(pkl_file):
        return pd.read_pickle(pkl_file)

    with open(f, 'r') as in_file:
        jsn = json.load(in_file)

    # extract the follower_ids and streamer_ids
    frst,snd = [], []
    for row in jsn:
        # use built in string regex to get the ideas out the 
        # json dictionary in string format
        temp = row['__key__']['path'].replace("\"", "")
        ids = temp.replace("\'", "").split(",")[1::2]
        frst.append(ids[0])
        snd.append(ids[1])

    # create a dataframe out of the follower_ids and streamer_ids
    data = np.vstack((frst,snd)).T
    df = pd.DataFrame(data,columns=cols).sort_values(cols[0])

    # rename columns to appropriate names
    df.rename(columns={'Streamer':'streamer_id', 
                    'Follower':'follower_id'},inplace=True)

    df.to_pickle(pkl_file)

    return df

In [0]:
def get_users_df(file_name="users"):
    """
        loads in user json file and converts it to pandas
    """
    f = clean_folder +  file_name +'.json'
    pkl_file = clean_folder +  file_name+'.pkl'

    # load if already exists
    if os.path.exists(pkl_file):
        return pd.read_pickle(pkl_file)

    # open json
    with open(f, 'r') as in_file:
        users = json.load(in_file)

    # dump __key__ dict out into user dict
    for user in users:
        if '__key__' in user:
            user.update(user.pop('__key__'))

    # get set of all keys
    cols = set()
    for user in users:
        for key in user:
            cols.add(key)

    #init data
    data = {}
    for col in cols:
        data[col] = []

    # extract data from users
    for user in users:
        for key in data:
            if key in user:
                data[key].append(user[key])
            else:
                data[key].append(0)

    # format data
    df_data = np.array([data[k] for k in data]).T

    # enumerate
    def enumm(x):
        if x == 'expert':
            return 4
        elif x == 'advanced':
            return 3
        elif x == 'intermediate':
            return 2
        elif x == 'beginner':
            return 1
        else:
            return 0

    enumm = np.vectorize(enumm)

    # export to pandas
    usersdf = pd.DataFrame(df_data, columns=list(data.keys()))
    usersdf = usersdf.set_index(usersdf['id'])
    usersdf.drop('id',axis=1,inplace=True)

    usersdf.learningLanguageProficiency = enumm(np.array(usersdf.learningLanguageProficiency.values))

    # divided columns into different types for efficient type conversions
    floats = ['version', 'followerCount','hopOnRate','points',
                'level','learningLanguageProficiency']
    dt = ['lastOnline']
    bools = ['online','streamer','admin','turnOffRingtone','suspended', 
                'receiveLevelUpPushNotifications',
                'receiveGroupChatInAppNotifications', 
                'receivePersonalChatInAppNotifications', 'banned', 
                'receiveLivestreamPushNotifications','darkMode']

    # convert to proper type
    for col in usersdf.columns:
        if col in floats:
            usersdf[col] = usersdf[col].astype('float')
        elif col in dt:
            usersdf[col] = pd.to_datetime(usersdf[col])
        elif col in bools:
            usersdf[col] = usersdf[col].astype('bool')
    
    usersdf.drop([],axis=1,inplace=True)

    # Rename columns to be more intuitive & drop unwanted columns
    cols = {'id':'user_id', 
            'lastStatusUpdateTime':'last_status_update', 
            'lastOnline':'last_online', 
            'nativeCountry':'native_country', 
            'receiveLivestreamPushNotifications':'livestream_push_notify', 
            'receivePushNotifications':'push_notify', 
            'hopOnRate':'hop_on_rate', 
            'learningLanguageProficiency':'english_proficiency', 
            'nativeLanguage':'native_lang', 
            'followerCount':'follower_count', 
            'coinBalance':'coin_balance'}

    usersdf.rename(columns=cols, inplace=True)

    # Rename the index to match other tables
    usersdf.index = usersdf.index.rename('user_id')
    
    # Drop columns for privacy and irrelevant factors
    usersdf.drop(columns=['firstName', 'lastName', 'userName', 'name',
                            'path','app','profilePicture','profilePictureOld',
                            'profilePictureIcon', 'profilePictureIconOld', 
                            'namespace','version','kind','online', 
                            'learningLanguage', 
                            'busy', 'darkMode', 'doNotDisturb', 'livestream', 
                            'turnOffRingtone', 'searchName', 
                            'receiveLevelUpInAppNotifications', 
                            'receiveFollowingPushNotifications', 
                            'receiveGroupChatPushNotifications', 
                            'receiveGroupChatInAppNotifications', 
                            'receiveLevelUpPushNotifications', 
                            'receivePersonalChatPushNotifications', 
                            'receivePersonalChatInAppNotifications', 
                            'receiveInAppNotifications', 
                            'receiveLivestreamInAppNotifications', 
                            'receiveFollowingInAppNotifications'], 
                inplace=True)

    # Replace 0's with NaN's and NaT's
    usersdf['native_country'] = usersdf['native_country'].replace(0, np.nan)
    usersdf['last_status_update'] =  pd.to_datetime(usersdf['last_status_update'].replace(0, pd.NaT))
    usersdf.loc[(usersdf.last_online == usersdf.iloc[0].last_online).values,'last_online'] = pd.NaT
    usersdf = usersdf[usersdf.index != 0]

    # Convert country abbreviations to full names
    def get_country_name(x):
        try:
            return countries.get(x).name
        except:
        # These are non-standard country codes
            if x == 'an':
                return 'Andorra'
            else:
                return x.capitalize()
    usersdf.native_country = usersdf.native_country[usersdf.native_country.notna()].apply(get_country_name)
    usersdf.native_country = usersdf.native_country.astype(str)

    # Convert language abbreviations to full names
    usersdf.native_lang = usersdf.native_lang.replace(0, np.nan)
    usersdf.native_lang = usersdf.native_lang[usersdf.native_lang.notna()].apply(lambda x: langcodes.get(x).language_name())

    # Convert Bio to participation flag
    usersdf['bio'] = usersdf['bio'].apply(lambda x: False if x in [0, '\n', ''] else True)

    # Convert other columns to proper data types
    usersdf.english_proficiency = usersdf.english_proficiency.astype(int)
    usersdf.follower_count = usersdf.follower_count.astype(int)
    usersdf.coin_balance = usersdf.coin_balance.astype(int)
    usersdf['push_notify'] = usersdf['push_notify'].astype(bool)

    # Save cleaned data to pickle file
    usersdf.to_pickle(pkl_file)
    # usersdf[usersdf['streamer'] == True].to_pickle(clean_folder + "streamers.pkl")
    # usersdf[usersdf['streamer'] == False].to_pickle(clean_folder + "non_streamers.pkl")

    return usersdf

In [0]:
def get_livestreams_df(file_name="livestream_recording"):
    """
    Loads in livestream recording data, and converts it to a pandas DataFrame
    """
    # Define the default files for working in colab.
    file = clean_folder + file_name + '.json'
    pkl_file = clean_folder + file_name +'.pkl'

    # Load the file if if already exists:
    if os.path.exists(pkl_file):
        return pd.read_pickle(pkl_file)

    # Open json file
    with open(file, 'r') as in_file:
        livestream_recording = json.load(in_file)

    # Find all the unique columns in the dataset
    unique_cols = set()
    bools = ['uploaded', 'hidden', 'premium', 'currentlyLive', 'uploaded']
    floats = ['views', 'startTimestamp']

    # Move data from '__key__' to livestream_recording dict
    for k in range(len(livestream_recording)):
        livestream_recording[k].update(livestream_recording[k].pop('__key__'))
        unique_cols.update(livestream_recording[k].keys())

    # Add nans where there is missing data
    for k in range(len(livestream_recording)):
        for key in unique_cols:
            if key not in livestream_recording[k].keys():
                if key in bools:
                    livestream_recording[k][key] = 0
                else:
                    livestream_recording[k][key] = np.nan

    # Cast our data as a dataframe
    lsr_df = pd.DataFrame(livestream_recording)

    # Extract non-redundant information about the streamer, then drop the column
    def get_fn(x):
        if isinstance(x,dict):
            return x.get('firstName')
        else: 
            return x

    lsr_df['streamerFirstName'] = lsr_df[lsr_df['streamer'].notna()].streamer.apply(get_fn)
    lsr_df.drop(columns='streamer', inplace=True)

    # Change the column datatypes 
    for column in lsr_df.columns:
        if column in bools:
            # Fill in nan values as False
            lsr_df[column] = lsr_df[column].astype(bool)
        elif column in floats:
            lsr_df[column] = lsr_df[column].astype(float)        
        else:
            lsr_df[column] = lsr_df[column].astype(str)
    
    # Convert the timestamp column to a datetime object
    lsr_df.startTimestamp = pd.to_datetime(lsr_df.startTimestamp, unit='ms')
    
    # Drop the unwanted columns
    cols_to_drop = lsr_df.columns & ['namespace', 'app', 'path', 'kind', 
                                     'filename', 'videoUrl', 'currentlyLive', 
                                     'thumbnail', 'agoraRecordingUid', 
                                     'agoraResourceId', 'agoraRecordingId']
    lsr_df.drop(columns=cols_to_drop, inplace=True)
    lsr_df.replace('nan', np.nan, inplace=True)

    # This is for a livestream table
    if 'streamerId' not in lsr_df.columns:
        lsr_df['streamerId'] = lsr_df.name
        lsr_df.drop('name',axis=1,inplace=True)

        # drop irrelevant columns
        ls.drop(columns=['streamerFirstName',
                         'autoRecord',
                         'recordingId'],inplace=True)

        # convert to proper types
        ls.viewers = ls.viewers.astype(int)

    # This is for livestream recording table
    else:
        lsr.rename(columns={'name':'stream_id'}, inplace=True)
        lsr.views = lsr.views.astype(int)

    lsr.rename(columns={'streamerId':'streamer_id', 
                        'startTimestamp':'start_time'}, inplace=True)
    
    lsr.drop(columns=['streamerFirstName'], inplace=True)
    
    lsr_df = lsr_df[lsr_df.streamerId.notna()]

    lsr_df.to_pickle(pkl_file)
    return lsr_df

In [0]:
def get_friends_df(file_name):
    """
    Loads in friends_table data, and converts it to a pandas DataFrame
    """
    # Define the default files for working in colab.
    file = clean_folder + file_name + '.json'
    pkl_file = clean_folder + file_name +'.pkl'
  
    # Load the file if if already exists:
    if os.path.exists(pkl_file):
        return pd.read_pickle(pkl_file)

    else:
        # Open json file
        with open(file, 'r') as in_file:
            friends = json.load(in_file)
        
        # get the unique columns for the friends table
        unique_cols = set()
        bools = []
        for k in range(len(friends)):
            friends[k].update(friends[k].pop('__key__'))
            unique_cols.update(friends[k].keys())

            # loop over the unique columns 
            for k in range(len(friends)):
                for key in unique_cols:

                    # set the dictionary appropriately 
                    if key not in friends[k].keys():
                        if key in bools:
                            friends[k][key] = 0
                        else:
                            friends[k][key] = np.nan
    

    # Cast our data as a dataframe & pickle it
    friends_df = pd.DataFrame(friends)
    friend_pattern = re.compile("\"[\w]+\", \"([\w]+)\", \"[\w]+\", \"[\w]+\"")
    friends_df['friend_id'] = friends_df['path'].apply(lambda x: friend_pattern.findall(x)[0])
    friends_df.rename(columns={'id':'user_id'}, inplace=True)

    # Drop columns for privacy & redundancy
    friends_df.drop(columns=['app', 'namespace', 'profilePicture', 
                                'kind', 'name', 'path', 'firstName'], inplace=True)
    friends_df.to_pickle(pkl_file)

    return friends_df

In [0]:
def loadpandas():
    """
    This loads all the cleaned json files into pandas, 
    saves them and returns them
    """
    # Run our function to get readable json files.
    clean_json_files()

    all_tables = {}

    all_tables['calls'] = get_calls_df('calls')
    all_tables['bans'] = get_bans_df('bans')
    all_tables['users'] = get_users_df()
    all_tables['lsr'] = get_livestreams_df("livestream_recording")
    all_tables['ls'] = get_livestreams_df("livestream")
    all_tables['friends'] = get_friends_df("friends")
    all_tables['followers'] = get_followers_df("followers",
                                               ["Streamer","Follower"])

    # Followed Livestreams and Followers appear to have similar information, 
    # (may have use for it in future)
    # all_tables['fl'] = get_followers_df("followed_livestream",
    #                                    ["Users","Followed_Livestreams"]) 
    return all_tables

In [0]:
tables = loadpandas()

In [0]:
#users table
tables['users'].sample(5)

,last_status_update,native_country,native_lang,points,streamer,follower_count,admin,coin_balance,bio,last_online,english_proficiency,banned,hop_on_rate,livestream_push_notify,suspended,push_notify,level
user_id,,,,,,,,,,,,,,,,,
Rs0uTm707DakujYzirmkpfMRP8F3,NaT,Viet Nam,Chinese,0.0,False,0,False,0,False,2019-10-01 09:21:41.567000+00:00,1,False,0.0,False,False,False,0.0
kjWrtwQVFyfhs6R66ZvSOdtjguw1,NaT,Malaysia,Malay,0.0,False,0,False,0,False,2019-11-13 17:30:25.998000+00:00,1,False,0.0,True,False,True,0.0
005c55t0uiXz0s91RL02ilDQNkN2,NaT,Viet Nam,English,0.0,False,0,False,0,False,NaT,2,False,0.0,False,False,False,0.0
BDjVk6E7GKRdT8QaMhiZsGOn3Dh1,NaT,Morocco,Arabic,0.0,False,0,False,0,False,NaT,2,False,0.0,False,False,False,0.0
onJvumDdAXZNcE2ob78Y0vYEkCx1,2019-06-19 17:49:07.933000+00:00,Morocco,Arabic,0.0,False,0,False,0,False,2019-06-19 17:49:07.933000+00:00,2,False,0.0,False,False,False,0.0


In [0]:
tables['users'][tables['users'].index == '008Z1Zo7FTOGEsI9CFnHeMKoeF03']

,last_status_update,native_country,native_lang,points,streamer,follower_count,admin,coin_balance,bio,last_online,english_proficiency,banned,hop_on_rate,livestream_push_notify,suspended,push_notify,level
user_id,,,,,,,,,,,,,,,,,
008Z1Zo7FTOGEsI9CFnHeMKoeF03,2019-08-14 12:08:18.933000+00:00,India,Hindi,0.0,False,0,False,0,True,NaT,1,True,0.0,False,False,False,0.0


In [0]:
# followers table
tables['followers']

,streamer_id,follower_id
23413,1Z6XYA1vgQXiTQTRpNhzXsj42Qk1,OA9PVrOYHkdPAm56vDtzncMHjL52
17847,1Z6XYA1vgQXiTQTRpNhzXsj42Qk1,IGKj8DE5ltUolZy6GCKvfMkZX5q2
49504,1Z6XYA1vgQXiTQTRpNhzXsj42Qk1,q662PDTGVIffYfYPLtp4W4OoMu13
36656,1Z6XYA1vgQXiTQTRpNhzXsj42Qk1,cJyMU6rTsieycSKpWzGHxRQ6eAu1
26566,1Z6XYA1vgQXiTQTRpNhzXsj42Qk1,RPMxeneyuIhhe9qQrJnD8QUmCbB2
...,...,...
1054,z8N27DEBoSUCjCticAaCzLSkjC22,17RqQClvHtcr0Ajwh1b2JX32YJy2
24505,z8N27DEBoSUCjCticAaCzLSkjC22,PAxOlcf267OEk5kYB9tdLjePBqf1
11373,z8N27DEBoSUCjCticAaCzLSkjC22,BvUM9ktm8bfX33oLsIpaoWDbtWA3
27070,z8N27DEBoSUCjCticAaCzLSkjC22,RyvqCZvQ0bMkkzyJVBUXWyRbHRk1


In [0]:
# live stream recordings table
tables['lsr'].sample(5)

,views,stream_id,premium,start_time,streamer_id,title,uploaded,category,hidden
90,2339,PueZCjtPkI5jxrs390yk,False,2019-09-11 03:52:22.722,71vksrN1EHancbxxtAnkhPI4VBj1,How to pronounce the 10 MOST COMMON WORDS in E...,True,Speaking,False
32,30,F5WPeQDlFI0tZGd4l7MK,False,2019-11-08 05:33:17.283,yOiAC2dilQPYLLoOJMa4v3dwLek2,testing,False,Culture,True
16,2,yuhFmqEItDXzQkfex9OL,False,NaT,NaN,NaN,False,NaN,False
49,1511,8JIukfdrBHB3OFCDcEhz,False,2019-08-27 17:40:01.831,JdopYYS74rQZIAjSAsRpwWBrfgl1,Culture topic: Let’s talk about families! 👨‍👩‍...,True,Culture,False
154,1732,4BBtzR2OvPr2n37hyjRc,False,2019-09-20 22:25:20.369,WWPLsmHyboSCClOJZPDR28AB9ct1,St. Lucia Honeymoon Part II,True,,False


In [0]:
#live streams
tables['ls'].sample(5)

,category,premium,title,start_time,viewers,streamer_id
1,Culture,False,TEST - resolving some issues,2019-08-14 21:52:50.663,13,EcPAITqb2fdG66Gz1EQQpyiqEkM2
19,Culture,False,testing again,2019-11-13 04:34:26.649,3,yOiAC2dilQPYLLoOJMa4v3dwLek2
18,Culture,False,Sunday Selection: Abhay - Nature + Food Idioms,2019-11-10 16:59:35.387,147,WWPLsmHyboSCClOJZPDR28AB9ct1
5,Culture,False,testing,2019-10-22 19:44:40.117,0,UxV75ClA1gZFNUySPJSm2GVqg243
8,Culture,False,100 Questions,2019-11-03 14:12:23.907,29,z8N27DEBoSUCjCticAaCzLSkjC22


In [0]:
# friends table
tables['friends'].sample(5)

,user_id,friend_id
57271,SEOTyCdlrFc1zZI2uYQggHd42e22,gqKHroIl2xTVu8zr88VuqXcvcwt1
325190,vf2tlq4EqhRDdyY7qzckpx8LnMP2,7PKXkPNT0XZfoZcIhc7bKQqgqrc2
60807,kBSTShIf5OhXnIAQZ5uL63QANSI3,WnaP4cDtiqQDPrp5TaB0QHzMBEu1
304682,usygtUYMc4cNtZXTBr3t8Y6CLcy1,MBsHWSKZVNdkrIsMLvxQyL6Fdqy2
76696,jx3UTcrKWTNJLri6ynVWfTYV66i1,Sa4B1yTO74TcxRyYZngTddrv78s1


In [0]:
# bans table
tables['bans']

,user_id,ban_id,description,vilolations
0,008Z1Zo7FTOGEsI9CFnHeMKoeF03,lNmT1N4tHsaEYS2BCtqt,bad person z,"hateful_conduct_and_harassmentz4, distributing..."
1,02MPFMtPUsZs9qvk2aDvGfXletN2,z3J3pwYVBrlMWaCbRO48,byeeez,+nudity_pornography_and_other_sexual_contentz
2,053x3c4VkxUmk0mOXfenNIAufHA2,3ByAwlbYQv600nK8OiVj,byeeeez,&spam_scams_and_other_malicious_conductz
3,054yvZffP2UrFUBJ4rVC7rNXWqy1,nLAO6qNwVh7eUQvszFxP,nan,"distributing_advertisementsz?, &spam_scams_and..."
4,09GJJt9c84PLBIR93MeAs9jmmc53,XE5Ub06HUWGw0VAbcpNj,bitcoinz,"&spam_scams_and_other_malicious_conductz4, dis..."
...,...,...,...,...
1574,zfuZVj3NBxbdI3HqIRfJVgPLuQ83,hPpeleT7UCBv8nC84kkS,byeeez,+nudity_pornography_and_other_sexual_contentz
1575,zjVNzej1xeVpvDlLDyQTRecDn7D2,oC02SNEU2okaRbT6xr8A,nan,+nudity_pornography_and_other_sexual_contentz
1576,zl77CIVCIpdY55CwiP7uUPDw4At2,zkMFZMyCq8muVAeZ8yRQ,nan,hateful_conduct_and_harassmentzn
1577,zlUWQz4e25OMElNuWwMkTHGSYe72,CK2uEk3YOWHwvzaTnytV,byeeeez,&spam_scams_and_other_malicious_conductz


In [0]:
# calls table
tables['calls'].sort_values(by="user_id")

,call_id,user_id,to_user_id,video,status
42,870ua3cyBeUDm2Jdy1oe,0M5C9cBfiuhXspMtbwhnKR4rbQw2z,dj8K09c8UcanZQ3i4vm1cMwVdmj2z,1,0
125,KOuAO9uJhltAaXM4lFX8,15mlRziqMzg8Evf2LYk2qw5Jxyt2z,6BF3i7IX0OSHyaIQjyNJwlGzLD62z,0,1
65,BeH0tTZtTf6ltVZTfrdp,2FRjyQo2VHWVOawPTU0GK7eBnLQ2z,AWZ9daFsisbctT6WT0m9iKuXX9U2z,1,0
74,Ca5R3E0bkIH3bGHR4BEx,3D3pt31jKTYozCRvQETNZeXipHX2z,FsT5zhML4GStl0uDCSNw9oE6z1o2z,1,1
31,6GKKHg85EtyriESiU6Xy,3qjlQiLuoIgOFSQ2pEsTMb0CpCp2z,KLoKL9uAblPyGfvNCKJ0TYbel2E2z,0,0
...,...,...,...,...,...
117,J2N0ygphgik4uJXym0u2,vyIJlsZi9Zdb41ZZmoOq1uV2PQi2z,X5JZVwOplwgumj5Aw7jBcxLiR9I3z,0,1
88,E4WdWMygerJCICvbQOed,wbvgDPxwKxP1VTumyTRYSNKCO7d2z,WTd22jgNeZaXot60yVMBx2J5vY73z,0,0
96,F2U6BPgcIq2CqAPbSzsl,wk4OnZKqMXYHHWj3a8fPMEjysev2z,2Qx5W2tKEphk0AdPwdMXN8uRYb52z,0,1
64,BOVOnIXVOVpMGardajIi,xwwHXXWvDWcy5lIE8yNmcoiooZx2z,WlNxvd4FMahtwICPWxyiMDzhh8m1z,0,1


In [0]:
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize
import requests.api
import re
from IPython.display import clear_output, display
from os import system, name 

def unpack_path_values(path_values):
    users_list = []
    conversations_list = []
    messages_list = []

    n = len(path_values)

    #unpack hidden fields inside of path column
    for i, text in enumerate(path_values):

        users_search = re.search(r'"users", "(.*?)"', text)
        conversations_search = re.search(r'"conversations", "(.*?)"', text)
        messages_search = re.search(r'"messages", "(.*?)"', text)

        if not users_search is None:
            users = users_search.group(1)
        else:
            users = np.nan

        if not conversations_search is None:
            conversations = conversations_search.group(1)
        else:
            conversations = np.nan

        if not messages_search is None:
            messages = messages_search.group(1)
        else:
            messages = np.nan

        users_list.append(users)
        conversations_list.append(conversations)
        messages_list.append(messages)

        print(f"rows to process: {n-i}")

    return users_list, conversations_list, messages_list

def clean_message_files():
    for i in range(10):
        df = pd.read_csv(f"Hallo_Data/messages/messages-{i}.csv")

        print("started unpacking values")
        users, conversations, messages = unpack_path_values(df['path'].values)
        print("finished unpacking values")

        #add unpacked fields to dataframe
        df["users"] = users
        df["conversation"] = conversations
        df["message"] = messages

        df = df.drop(columns=["path"])

        print("started pickling")
        #save results
        df.to_pickle(f"Hallo_Data/messages_clean/messages-{i}.pkl")
        print("finished pickling")

def stack_message_files():
    df_0 = pd.read_pickle("Hallo_Data/messages_clean/messages-0.pkl")

    for i in range(1, 10):
        print(f"concatenating {i}")
        df_1 = pd.read_pickle(f"Hallo_Data/messages_clean/messages-{i}.pkl")
        df_0 = pd.concat([df_0, df_1], ignore_index=True)

    df_0.to_pickle(f"Hallo_Data/messages_clean/messages.pkl")



stack_message_files()



def format_data():
    with open("Hallo_Data/messages-0.json", 'r+') as infile:
        text = infile.readlines()[-1]
        inner_text = re.findall(r'\w+\\"', text)
        users = inner_text[1][:-2]
        conversations = inner_text[3][:-2]
        messages = inner_text[5][:-2]
        # print(inner_text)
        # print("\n")
        # print(users, conversations, messages)
        # print("\n")
        # print(text)
        
        return
        text = '[' + ",".join(text) + ']'

    with open("Hallo_Data/test1.json", 'r+') as infile:
        infile.write(text)

def clean_message_data():
    df = pd.read_json("Hallo_Data/test1.json")
    print(df[:3].values)

In [0]:
m = pd.read_pickle(clean_folder+"messages/messages-0.pkl")

In [0]:
m.sample(5)

,messageType,timestamp,duration,users,conversation,message
4221578,text,1569082197055,0,4Q7pFbKCzLMofgQNd0gPDXxehO93,4Q7pFbKCzLMofgQNd0gPDXxehO93_KcNHzpOIv2SvkrivK...,VwcoFkCufkRUHqIPPiz4
1277993,text,1572009561595,0,0pMgLQVmqpOoTSgRigtzMmA30QP2,0pMgLQVmqpOoTSgRigtzMmA30QP2_QpBzzS05XqSAkkoJ6...,rzNdjCcaUBbJPN7KH5jI
4610201,text,1563449589490,0,Vz9QjmmuBqPQfKOOwUDPhVoFzx73,Vz9QjmmuBqPQfKOOwUDPhVoFzx73_qVds125unCS4xngUI...,CI8O2djhgZIYR6kjjIRd
1900840,text,1567083540122,0,eNY4FFic16TS3b8hUyeOfXp4nw12,3nlhYEb3RNPILo4Mj5swSwcuMLf2_eNY4FFic16TS3b8hU...,LByIhWjV0bSo5ymp5T0N
2731416,text,1570576100692,0,hjomZO0XXnXE8rAxIbhaHlte1Bp2,NT6BwVwOoqNA9FGasi5gPzDt5pp2_hjomZO0XXnXE8rAxI...,A71Z3JOG0RsyJxCJw4hu


In [0]:
pd.read_pickle(clean_folder+"messages/messages-1.pkl")

,messageType,timestamp,duration,users,conversation,message
0,text,1563745896920,0,GhiR9zmhPnaV0wq655PxyZsVtAg2,GhiR9zmhPnaV0wq655PxyZsVtAg2_fFWV566rxJdY4fm5q...,mSMH0q6POFMj0hz6fR4D
1,text,1563998851338,0,GhiR9zmhPnaV0wq655PxyZsVtAg2,GHCWuZlKCmS7xNtBDY3Hg3yiRzl1_GhiR9zmhPnaV0wq65...,U23yqKj4ktDWSdZ5IK0W
2,text,1569668424140,0,S2Agtp6BKIOphYbOeLaCTnOuKuu1,1BIplSsRJOdY6YyPiAg1wgQICed2_S2Agtp6BKIOphYbOe...,0eXwTl1VgR8NAXfn7TW1
3,text,1569900194977,0,S2Agtp6BKIOphYbOeLaCTnOuKuu1,IvUA0oUAzwYGkzJhGOnNObfzyId2_S2Agtp6BKIOphYbOe...,dKRLinilMgJUINUxTeYJ
4,text,1567328751439,0,tgHERuLkSVMS7Hu0mB1ayLyO2f12,XvkzrSuX0sZlR40xuuydkSFQeeE3_tgHERuLkSVMS7Hu0m...,Y0jyNcJtP4RTcHKmeFdG
...,...,...,...,...,...,...
5096148,text,1561971980313,0,67fmXQbqDYQIJsXyYQvx99qQ2KI3,67fmXQbqDYQIJsXyYQvx99qQ2KI3_JTnoRQSsSLWcVnfWR...,pj4MfnsLabfO9W1JN0OI
5096149,text,1571166068035,0,2ArS6jsm9YeJFxFbYPSbIQmKxT13,2ArS6jsm9YeJFxFbYPSbIQmKxT13_JTnoRQSsSLWcVnfWR...,qmYmv7uBWOQwUVDnKTZn
5096150,text,1571394936625,0,JTnoRQSsSLWcVnfWRy52EucSf8T2,8GlCpFcaaMaD9kVsKAKs4aIExls2_JTnoRQSsSLWcVnfWR...,tAACZYv6YcFe5BbzmUTC
5096151,text,1571146342540,0,lctF5xG4KlQAlk3fqO68D1h1lQn2,JTnoRQSsSLWcVnfWRy52EucSf8T2_lctF5xG4KlQAlk3fq...,i4YQw1MmGGas8UWNEUCA
